In [1]:
import os
os.chdir("..")
print(os.getcwd())

/Users/andreaspletschko/Uni/5. Semester (HKUST)/Machine_Learning/COMP5212


In [2]:
MODEL_PATH = "checkpoints/model_nrg01.hpc.itc.rwth-aachen.de_20231123-201621_mobilenet_v3_large_0_unfreezed_20_epochs.pth"
TSV_PATH = "data/kaggle_dataset/train.tsv"
TEST_TSV_PATH = "data/kaggle_dataset/test.tsv"
IMAGE_PATH = "notebooks/demo_images/1.png"

In [3]:
from PIL import Image
import torch
from torchvision import transforms
import numpy as np

/Users/andreaspletschko/.local/lib/python3.11/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.18) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [4]:
from src.models.model import get_model
import src.kaggle50k_dataset as kaggle50k_dataset

In [5]:
def load_model(model_path):
    state_dict = torch.load(model_path, map_location=torch.device('cpu'))
    model_state_dict = state_dict["model_state_dict"]
    idx_to_label = state_dict["idx_to_labels"]
    config = state_dict["config"]
    model = get_model(model_name=config["model"]["name"], num_classes=56)
    model.load_state_dict(model_state_dict)
    model.eval()
    return model, idx_to_label, config

def get_idx_to_label(tsv_path):
    labels = []
    tsv_file = open(tsv_path, "r")

    for line in tsv_file:
        if line in ['\n', '\r\n']:
            continue
        label, _ = os.path.split(line.strip())
        labels.append(label)

    idx_to_label = {idx: label for idx, label in enumerate(set(labels))}
    return idx_to_label

def load_img(image_path):
    transform = transforms.Compose([
        transforms.Resize((1536, 662)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    image = Image.open(image_path).convert("RGB")
    image_transformed = transform(image)
    return image_transformed

def get_random_image(dataset):
    idx = np.random.randint(0, len(dataset))
    image, label = dataset[idx]
    return image, label

In [6]:
model, idx_to_label, config = load_model(MODEL_PATH)
label_to_idx = {label: idx for idx, label in idx_to_label.items()}
test_dataset = kaggle50k_dataset.Kaggle50K(TEST_TSV_PATH)
test_dataset.labels_to_idx = label_to_idx

img, label = get_random_image(test_dataset)
img.show()

/Users/andreaspletschko/opt/miniforge3/envs/ml_project/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/andreaspletschko/opt/miniforge3/envs/ml_project/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Large_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Large_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
img_transformed = kaggle50k_dataset.transform(img).unsqueeze(0)

with torch.no_grad():
    output = model(img_transformed)
    # make probabilities
    output = torch.nn.functional.softmax(output, dim=1)
    # top 3
    probabilities, predicted = torch.topk(output.data, 5, dim=1)
    predicted = predicted.squeeze(0)
    print(probabilities)
    print(idx_to_label[predicted[0].item()])
    print(idx_to_label[predicted[1].item()])
    print(idx_to_label[predicted[2].item()])
    print(idx_to_label[predicted[3].item()])
    print(idx_to_label[predicted[4].item()])

tensor([[0.2272, 0.2114, 0.2012, 0.0706, 0.0629]])
data/kaggle_dataset/United States
data/kaggle_dataset/France
data/kaggle_dataset/United Kingdom
data/kaggle_dataset/Netherlands
data/kaggle_dataset/Norway


In [8]:
img.show()
print(idx_to_label[label.item()])

data/kaggle_dataset/France
